In [1]:
# Transformers installation
! pip install transformers datasets
# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00


In [2]:
! pip install -U accelerate
! pip install -U transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 22.4 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


# Fine-tune a pretrained model

사전 학습된 모델을 사용하면 상당한 이점이 있습니다. 계산 비용과 탄소 배출량을 줄이고 처음부터 훈련하지 않고도 최신 모델을 사용할 수 있습니다. 🤗 Transformers는 다양한 작업을 위해 수천 개의 사전 훈련된 모델에 대한 액세스를 제공합니다. 사전 훈련된 모델을 사용하는 경우 작업에 특정한 데이터 세트에서 모델을 훈련합니다. 이 튜토리얼에서는 선택한 딥러닝 프레임워크로 사전 학습된 모델을 미세 조정합니다.

* Fine-tune a pretrained model with 🤗 Transformers [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer).
* Fine-tune a pretrained model in TensorFlow with Keras.

<a id='data-processing'></a>

## Prepare a dataset

사전 훈련된 모델을 미세 조정하기 전에 데이터 세트를 다운로드하고 훈련을 위해 준비하세요.

Begin by loading the [Yelp Reviews](https://huggingface.co/datasets/yelp_review_full) dataset:

In [3]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")
dataset["train"][100]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

텍스트를 처리하고 가변 시퀀스 길이를 처리하기 위한 패딩 및 truncation 전략을 포함하려면 토크나이저가 필요합니다.

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Map:   0%|          | 0/650000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

원하는 경우 전체 데이터 세트의 더 작은 하위 집합을 만들어 미세 조정하여 소요 시간을 줄일 수 있습니다.

In [13]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

<a id='trainer'></a>

## Train

🤗 Transformers는 교육에 최적화된 [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) 클래스를 제공합니다. 자신의 훈련 루프. [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) API는 로깅, 기울기 누적, 혼합 정밀도와 같은 다양한 훈련 옵션과 기능을 지원합니다.

모델을 로드하여 시작하고 예상 레이블 수를 지정합니다. Yelp Review [데이터 세트 카드](https://huggingface.co/datasets/yelp_review_full#data-fields)에서 5개의 레이블이 있음을 알 수 있습니다.

In [14]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Training hyperparameters

다음으로 조정할 수 있는 모든 하이퍼파라미터와 다양한 훈련 옵션을 활성화하기 위한 플래그가 포함된 [TrainingArguments](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.TrainingArguments) 클래스를 만듭니다.

이 튜토리얼의 경우 기본 [하이퍼파라미터](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments)로 시작할 수 있지만 자유롭게 실험하여 최적의 설정을 찾으십시오.

교육에서 체크포인트를 저장할 위치를 지정합니다.

In [15]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

### Metrics

[트레이너](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer)는 트레이닝 중에 모델 성능을 자동으로 평가하지 않습니다. 메트릭을 계산하고 보고하려면 [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer)에 함수를 전달해야 합니다. 🤗 데이터셋 라이브러리에서는 `load_metric`(자세한 내용은 이 [튜토리얼](https://huggingface.co/docs/datasets/metrics.html) 함수를 참조하세요) 함수와 함께 로드할 수 있는 간단한 [`accuracy`](https://huggingface.co/metrics/accuracy) 함수를 제공합니다:

In [16]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


`metric`에서 `compute`를 호출하여 예측의 정확도를 계산합니다. 계산`에 예측을 전달하기 전에 예측을 로그로 변환해야 합니다(모든 🤗 트랜스포머 모델은 로그를 반환한다는 점을 기억하세요):

In [17]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

미세 조정 중에 평가 지표를 모니터링하려면 훈련 인수에 `evaluation_strategy` 매개 변수를 지정하여 각 에포크가 끝날 때 평가 지표를 보고하세요:

In [18]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

### Trainer

모델, 훈련 인수, 훈련 및 테스트 데이터 세트, 평가 함수를 사용하여 [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) 개체를 만듭니다:

In [19]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

Then fine-tune your model by calling [train()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.train):

In [20]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.140377,0.515000
2,No log,0.998967,0.581000
3,No log,1.015947,0.608000


TrainOutput(global_step=375, training_loss=0.964901123046875, metrics={'train_runtime': 374.626, 'train_samples_per_second': 8.008, 'train_steps_per_second': 1.001, 'total_flos': 789354427392000.0, 'train_loss': 0.964901123046875, 'epoch': 3.0})

<a id='keras'></a>

🤗 트랜스포머 모델은 Keras API를 통해 텐서플로우의 트레이닝도 지원합니다.

### Convert dataset to TensorFlow format

[DefaultDataCollator](https://huggingface.co/docs/transformers/main/en/main_classes/data_collator#transformers.DefaultDataCollator)는 모델이 학습할 텐서를 일괄 처리로 조립합니다. 텐서플로 텐서를 반환하려면 `return_tensors`를 지정해야 합니다:

In [21]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator(return_tensors="tf")

<Tip>

[트레이너](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer)는 기본적으로 [DataCollatorWithPadding](https://huggingface.co/docs/transformers/main/en/main_classes/data_collator#transformers.DataCollatorWithPadding)을 사용하므로 데이터 콜레이터를 명시적으로 지정할 필요가 없습니다.

</Tip>

다음으로, [`to_tf_dataset`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Dataset.to_tf_dataset) 메서드를 사용하여 토큰화된 데이터셋을 텐서플로 데이터셋으로 변환합니다. 입력은 `columns`에, 레이블은 `label_cols`에 지정합니다:

- Token ID -> 정답 데이터 만드는데 사용


- Python: List
- Numpy: nparray
- DL (TF, Torch): tensor -> tf.tensor, torch.tensor

In [22]:
tf_train_dataset = small_train_dataset.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

tf_validation_dataset = small_eval_dataset.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

/usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:400: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(


### Compile and fit

예상 레이블 수로 TensorFlow 모델을 로드합니다:

In [23]:
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


그런 다음 다른 Keras 모델과 마찬가지로 [`fit`](https://keras.io/api/models/model_training_apis/)을 사용하여 모델을 컴파일하고 미세 조정합니다:

In [24]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=tf.metrics.SparseCategoricalAccuracy(),
)

model.fit(tf_train_dataset, validation_data=tf_validation_dataset, epochs=3)

Epoch 1/3


ResourceExhaustedError: Graph execution error:

Detected at node tf_bert_for_sequence_classification/bert/encoder/layer_._11/attention/self/MatMul defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/iol

<a id='pytorch_native'></a>